Here we analyze one of the past votes from uma.

https://vote.uma.xyz/

"Will Tim Walz say "gun" during the debate?"

Polymarket | Vote number #369

We fetched the relevant description and discussions as a txt file for a quick demo. 
This file is located at: `assets/data/v001/discussions.txt`


In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
filepath = "../assets/data/v001/discussions.txt"
txt = open(filepath).read()
txt

'Discussion\nbruna.uma 2024-10-02, 4:59 PM\nDescription: Tim Walz and J.D. Vance are currently scheduled to debate on October 1, in an event hosted by CBS.\n\nThis market will resolve to "Yes" if Walz says "gun" at any point during the first vice presidential debate between Tim Walz and J.D. Vance in the 2024 election cycle. Otherwise, this market will resolve to "No".\n\nAny usage of the word regardless of context will count toward the resolution of this market.\n\nPluralization/possessive of the word will count toward the resolution of this market, however other forms will NOT count.\n\nCompound words will count as long as "gun" is part of the compound word and references the meaning which refers to a weapon designed to discharge bullets.\n\nIf no such debate in the 2024 election cycle happens by November 4, 2024, 11:59 PM ET, or if either candidate drops out of the race, this market will resolve to "No".\n\nThe resolution source will be the video of the debate. res_data: p1: 0, p2: 

In [3]:
from truth.tools.discussion_parser import DiscussionParser

# lets properly parse this txt file using Mistral's API
parser = DiscussionParser()

2024-10-06 08:31:50.019 | INFO     | truth.tools.discussion_parser:__init__:26 - DiscussionParser initialized with model: mistral-small-latest


In [4]:
parsed = parser.parse_from_file(filepath)
parsed

{'description': 'Tim Walz and J.D. Vance are currently scheduled to debate on October 1, in an event hosted by CBS.\n\nThis market will resolve to "Yes" if Walz says "gun" at any point during the first vice presidential debate between Tim Walz and J.D. Vance in the 2024 election cycle. Otherwise, this market will resolve to "No".\n\nAny usage of the word regardless of context will count toward the resolution of this market.\n\nPluralization/possessive of the word will count toward the resolution of this market, however other forms will NOT count.\n\nCompound words will count as long as "gun" is part of the compound word and references the meaning which refers to a weapon designed to discharge bullets.\n\nIf no such debate in the 2024 election cycle happens by November 4, 2024, 11:59 PM ET, or if either candidate drops out of the race, this market will resolve to "No".\n\nThe resolution source will be the video of the debate. res_data: p1: 0, p2: 1, p3: 0.5. Where p1 corresponds to No, 

In [5]:
# parsed is a dict with keys: 'description', 'messages'
parsed.keys()

dict_keys(['description', 'messages'])

In [6]:
parsed["messages"][0]

{'user': 'bowtune_18048',
 'timestamp': '2024-10-03, 5:37 PM',
 'P': 'P2',
 'evidence': 'mentioned at 1:02:21 https://www.youtube.com/live/VAGZGQg31hs?si=cpx5wfmr9r45g0_H&t=3741',
 'sources': ['https://www.youtube.com/live/VAGZGQg31hs?si=cpx5wfmr9r45g0_H&t=3741'],
 'rationale': ''}

In [7]:
# lets see the metrics, quantitative info about the votes
parser.calculate_metrics()

{'total_messages': 6,
 'unique_users': 6,
 'p_value_distribution': {'P1': 0, 'P2': 1, 'P3': 0, 'P4': 3},
 'evidence_count': 2,
 'evidence_percentage': 33.33333333333333,
 'evidence_with_links_count': 1,
 'evidence_with_links_percentage': 50.0,
 'total_links': 1,
 'rationale_count': 2,
 'rationale_percentage': 33.33333333333333,
 'time_span': '1 day, 0:36:00',
 'most_common_p_value': ('P4', 3),
 'resolution_status': 'Unresolved'}

In [8]:
# let's write the results for later use
parser.write()

2024-10-06 08:32:10.692 | INFO     | truth.tools.discussion_parser:write:201 - Parsed discussion written to: ../assets/data/v001/parsed_discussions.json
2024-10-06 08:32:10.693 | INFO     | truth.tools.discussion_parser:write:202 - Metrics written to: ../assets/data/v001/metrics.json


In [9]:
# now lets verify the votes
from truth import VerifierAgent
agent = VerifierAgent(model = "mistral-large-latest") # due to expected context size

2024-10-06 08:32:15.119 | SUCCESS  | truth.verifier:__init__:34 - Verifier Agent initialized with model: mistral-large-latest


In [11]:
res = agent.verify_uma_vote(contract_description=parsed["description"], 
                            message=parsed["messages"][0])


2024-10-06 08:32:41.930 | DEBUG    | truth.verifier:plan_actions:103 - Action descriptions: - read_wiki_entry: Reads the summary of a Wikipedia entry from a given Wikipedia URL.
- read_youtube_transcript: Retrieves the transcript of a YouTube video from a given YouTube URL. Applicable only to YouTube links.
- read_webpage_content: Fetches and extracts the text content from a webpage URL.


2024-10-06 08:32:45.853 | INFO     | truth.verifier:log_event:54 - Event: Plan Actions
Input: {'question': 'Fetch more context from the sources', 'links': ['https://www.youtube.com/live/VAGZGQg31hs?si=cpx5wfmr9r45g0_H&t=3741']}
Output: [{'action_name': 'read_youtube_transcript', 'params': {'url': 'https://www.youtube.com/live/VAGZGQg31hs?si=cpx5wfmr9r45g0_H&t=3741'}, 'reason': 'The link is a YouTube live video, and retrieving the transcript will provide the necessary context.'}]
---
2024-10-06 08:32:45.854 | INFO     | truth.verifier:take_action:160 - Taking action: read_youtube_transcript with params: {'url': 'https://www.youtube.com/live/VAGZGQg31hs?si=cpx5wfmr9r45g0_H&t=3741'}
2024-10-06 08:32:46.980 | INFO     | truth.verifier:log_event:54 - Event: Action Taken: read_youtube_transcript
Input: {'url': 'https://www.youtube.com/live/VAGZGQg31hs?si=cpx5wfmr9r45g0_H&t=3741'}
Output: {'success': True, 'content': 'tonight with just over a month to go until election day the first and only 

In [12]:
res

{'voted_P': 'P2',
 'is_correct': True,
 'confidence': 'High',
 'explanation': "The contract description clearly states that the market will resolve to 'Yes' if Walz says 'gun' during the first vice presidential debate. The provided evidence includes a transcript of the debate where Walz discusses gun-related issues, which meets the contract criteria. Therefore, the submitted P value of P2 is correct."}

In [13]:
# lets check the transcript to see if Tim Walz said "gun"
youtube_transcript=agent.context["read_youtube_transcript"][0]["content"]
youtube_transcript


"tonight with just over a month to go until election day the first and only meeting between the two men who hope to become Vice President of the United States Democratic governor Tim Walls of Minnesota and Republican senator JD Vance of Ohio now this is likely the final debate of this election cycle and voting is already underway in 20 States CBS News polling shows this remains a race either presidential candidate could win the CBS News vice presidential debate Starts Now fight for every every single vote and we're going to take this country back and we are ready to continue to build the future together we're going to turn this whole country read with President Donald J Trump's leadership you know what at the end of this little journey kamla Harris as the next president of the United States this is a CBS News special live from CBS News headquarters in New York America decides the vice presidential debate [Music] good evening I'm Nora odonnell and thank you for joining us for tonight's 

In [14]:
# Find occurrences of "gun" in youtube_transcript
gun_occurrences = youtube_transcript.lower().count("gun")
print(f"The word 'gun' appears {gun_occurrences} times in the transcript.")

# Find the index of the first occurrence of "gun"
first_occurrence_index = youtube_transcript.lower().find("gun")

if first_occurrence_index != -1:
    # Extract a snippet of context around the first occurrence
    start = max(0, first_occurrence_index - 50)
    end = min(len(youtube_transcript), first_occurrence_index + 50)
    context = youtube_transcript[start:end]
    print(f"\nContext of first occurrence:\n{context}")
else:
    print("\nThe word 'gun' was not found in the transcript.")

# to verify check the following youtube link :)
# https://www.youtube.com/live/VAGZGQg31hs?t=3741s


The word 'gun' appears 26 times in the transcript.

Context of first occurrence:
sidential debate we want to turn now to America's gun violence epidemic the leading cause of death f
